## Initializing the Local Model

In [2]:
from langchain_community.llms import Ollama 
llm = Ollama(model="llama3", temperature = 0)

In [3]:
llm.invoke("What is LangGraph?")

'LangGraph is a graph-based language model that represents languages as graphs, where nodes represent words or subwords and edges represent relationships between them. The goal of LangGraph is to capture the complex structure and patterns in natural language data, such as syntax, semantics, and pragmatics.\n\nIn traditional language models, text is typically represented as a sequence of tokens (e.g., words or characters), which can be processed using recurrent neural networks (RNNs) or transformers. However, these models may not fully capture the intricate relationships between words in a sentence or document.\n\nLangGraph addresses this limitation by representing languages as graphs, where each node corresponds to a word or subword, and edges represent various relationships between them, such as:\n\n1. Syntactic dependencies: The graph captures the hierarchical structure of sentences, including subject-verb-object relationships.\n2. Semantic relationships: Edges can represent semantic

## Indexing: Load

In [28]:
from langchain_community.document_loaders import WebBaseLoader 

loader = WebBaseLoader(
    web_path=("https://isqua.org/blog/covid-19/covid-19-blogs.html"),
)
docs= loader.load()

In [29]:
docs[0].page_content

"\n\n\n\n\n\n\n\n\n\n\n\n\n\nCOVID-19 Blogs\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHomeScholarshipsLucian Leape Patient Safety FellowshipAboutPEOPLEAWARDSPARTNERSResources & BlogBLOGRESOURCESIJQHC JOURNALIJCOMS JOURNALIKAS DATAContact UsPARTNERSHIP WITH PATIENTS\n\n\n\n\n\n\n\n\nToggle navigation\n\n\n\n\n \n\n\nMEMBERSHIPINSTITUTIONAL MEMBERSIJQHC JOURNALIJCOMS JOURNALEVENTSISTANBUL 2024 INTERNATIONAL CONFERENCECONFERENCE PROGRAMMEPLENARY SPEAKERSDELEGATE INFORMATIONEDUCATIONAL SITE VISITSPRE & POST CONFERENCE TOURSPAST CONFERENCESUPCOMING EVENTS & CONFERENCESWORLD PATIENT SAFETY DAYWPSD 2023EXTERNAL EVALUATION - IEEAEDUCATIONFellowship ProgrammeMiddle East Fellowship Specialist CertificatesIntroduction to QI & PSFellowship AmbassadorsNETWORKSCOMMUNITIES OF PRACTICELatin AmericaFrench SpeakingISQua ACADEMYISQua EXPERT\n\n\n\n\n\nHomeScholarshipsLucian Leape Patient Safety FellowshipAboutPEOPLEAWARDSPARTNERSResources & BlogBLOGRESOURCESIJQHC JOURNALIJCOM

In [30]:
len(docs[0].page_content)

20304

## Indexing: Split

In [31]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter= RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
    add_start_index = True,
)

In [32]:
all_splits = text_splitter.split_documents(docs)

In [33]:
len(all_splits)

29

In [34]:
len(all_splits[1].page_content)

849

## Indexing: Store

In [35]:
from langchain_community.embeddings import OllamaEmbeddings

embedding = OllamaEmbeddings(
    model="nomic-embed-text",
)

In [36]:
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents = all_splits,
    embedding = embedding,
)

In [37]:
retriever = vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k":6}
)

In [38]:
retriever.get_relevant_documents(
    "How to handle covid 19?"
)

[Document(page_content='b) Establish effective patient flow at facilities including screening, triage and targeted referral of COVID-19 and non-COVID-19 cases.\nc) Reduce transmission by isolating cases from other patients (or at least cohorting), and minimise the number of staff caring for each patient.\n\xa0\n3. Care of COVID-19 patients\na) Initiate IPC at point of entry to hospital. Immediately screen and isolate COVID-10 suspects from other patients. If illness is mild, immediately discharge for self-isolation at home.\nb) Assess whether patients will benefit from admission or transfer from other facilities. Consider not admitting patients with respiratory failure if the hospital is not equipped. This is equally efficacious, more humane for the patient, and reduces the risk of spread of infection to staff and other patients.\nc) Establish guidance for handling deceased patients.\nd) Minimise presence of visitors and non-essential staff.\n\xa0\n4. Infection prevention and control (

## Retrieval and Generation: Generate

In [39]:
from langchain import hub 

prompt= hub.pull("rlm/rag-prompt")

In [40]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [41]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt 
    | llm 
    | StrOutputParser()
)

In [42]:
rag_chain.invoke("How to handle covid 19?")

'To handle COVID-19, it is essential to establish effective patient flow at facilities, including screening, triage, and targeted referral of COVID-19 and non-COVID-19 cases. Additionally, reduce transmission by isolating cases from other patients (or at least cohorting), and minimize the number of staff caring for each patient.'

In [43]:
rag_chain.invoke("What is covid 19?")

"Based on the provided context, COVID-19 is a global pandemic caused by the coronavirus that has severe economic and financial consequences on people's lives. It has become more than just a health crisis for all countries with critical social, economic, and political consequences."

Thanks for reading 😀

Let's connect [YouTube](http://youtube.com/tirendazacademy) | [Medium](http://tirendazacademy.medium.com) | [X](http://x.com/tirendazacademy) | [Linkedin](https://www.linkedin.com/in/tirendaz-academy) 😎